# Building a TF Model
Trying some code snippets from François Chollet's "Deep Learning with Python", Chapter 3.

In [ ]:
import tensorflow as tf
from tensorflow import keras

## Building Custom Layer using `keras.layers.Layer`
Standard affine transformation `y = W*X+b`

In [11]:
class SimpleDense(keras.layers.Layer):
    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units
        self.activation = activation
    
    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.W = self.add_weight(
            shape=(input_dim, self.units), 
            initializer="random_normal")
        self.b = self.add_weight(
            shape=(self.units,),
            initializer="zeros")
    
    def call(self, inputs):
        y = tf.matmul(inputs, self.W) + self.b
        if self.activation is not None:
            y = self.activation(y)
        return y

In [12]:
my_dense = SimpleDense(units=32, activation=tf.nn.relu)
input_tensor = tf.ones(shape=(2, 784))
output_tensor = my_dense(input_tensor)

print(output_tensor.shape)

(2, 32)


In [13]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Dense(32, activation="relu"),
    layers.Dense(32)
])

## Automatic Shape Inference in TF
```python
def __call__(self, inputs):
    if not self.built:
        self.build(inputs.shape)
        self.built = True
    return self.call(inputs)
```

In [15]:
model = keras.Sequential([
    SimpleDense(32, activation="relu"),
    SimpleDense(64, activation="relu"),
    SimpleDense(32, activation="relu"),
    SimpleDense(10, activation="softmax")
])

## Sample Compilation Step
Metrics are for analysis and are not optimized. They don't need to be differentiable.
### Some default optimizers:
- SGD (with or without momentum)
- RMSprop
- Adam
- Adagrad

### Some default losses:
- CategoricalCrossentropy
- SparseCategoricalCrossentropy
- BinaryCrossentropy
- MeanSquaredError
- KLDivergence
- CosineSimilarity

### Metrics:
- CategoricalAccuracy
- SparseCategoricalAccuracy
- BinaryAccuracy
- AUC
- Precision
- Recall

### Customizing compilation 
```python
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-4),
              loss=my_custom_loss,
              metrics=[my_custom_metric_1, my_custom_metric_2])
```

In [19]:
model.compile(optimizer="rmsprop", 
              loss="mean_squared_error",
              metrics=["accuracy"])
# Equivalent
# model.compile(optimizer=keras.optimizers.RMSprop(), 
              # loss=keras.losses.MeanSquaredError(),
              # metrics=[keras.metrics.BinaryAccuracy()])

## Fitting your model with `fit()`
Pass in
- Data: inputs and targets
    - Either numpy arrays or TF `Dataset` object
- Epochs to train for
- Batch size with each epoch of mini-batch GD

```python
history = model.fit(
    inputs,
    targets,
    epochs=5,
    batch_size=128
)
```

History objects, which is a dict mapping keys (e.g. "loss") or specific metric names to the list of per-epoch values

### Validation
Use validation sets to see how data is doing per epoch on values your model hasn't seen before. 
Pass in the `validation_data=(val_inputs, val_targets` within `model.fit()`

Seeing validation loss and metrics after training is complete, you can use `evaluate()`

```python
loss_and_metrics = model.evaluate(val_inputs, val_targets, batch_size=128)
```

## Model Inference
I.E. predictions on new data. Use `__call__()` method, which is called by default using the variable name.

```python
predictions = model(new_inputs)
```
However, this tries to process all inputs at once, which may not be feasible depending on your processing power. 
We can use `predict()` instead. The batch_size decides how many it does at once)

```python
predictions = model.predict(new_inputs, batch_size=128)
```